[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1tyAfk10dowfjXP3bY9s0sRLVETYFdgDI)

Uncomment the following line to install [geoca](https://pypi.org/project/geoca/), [tqdm](https://pypi.org/project/tqdm/), and [pymannkendall](https://pypi.org/project/pymannkendall/) if needed.

In [ ]:
# !pip install geoca tqdm pymannkendall

Import modules: **rasterconvert**, **tqdm**, and **pymannkendall**.

Use **rasterconvert** to convert the raster data format, **tqdm** to create a progress bar when running the program, and **pymannkendall** to perform MK trend tests.

In [ ]:
from geoca import rasterconvert
from tqdm import tqdm
import pymannkendall as mk

**rasterconvert.multiRaster_listDict(folder_path, index_list)**:

- Reads multiple raster data (.tif) from a folder and stores it in a dictionary based on specific index values.

The example data is the precipitation raster data of a region from 1961-2022, read the data of each year by multiRaster_listDict function, and input the raster into the data dictionary in the form of list.

In [ ]:
index_list = list(range(1961, 2022 + 1))
data_dict = rasterconvert.multiRaster_listDict("../Pre_Year_NE", index_list)

In [ ]:
# Output the list of 2011 data in the result dictionary line by line
# Example of the list of 2011 data: [[None, 1, 2],[4, 5, 3],[7, 6, None]]
for row in data_dict[2011]:
    print(row)

**rasterconvert.reorganize_multiRaster_listDict(data_dict, index_list)**:

- Reorganizes data from a list dictionary representing multiple raster data on the basis of the multiRaster_listDict function.

In [ ]:
data_index_list, data_value_dict = rasterconvert.reorganize_multiRaster_listDict(data_dict, index_list)

In [ ]:
# Output data_index_list list line by line
# The list is a two-dimensional indexed list with the same number of rows and columns as the original raster data, and the list element values are tuples of horizontal and vertical coordinates
# Example of the data_index_list list: [[(1, 1), (1, 2), (1, 3)],[(2, 1), (2, 2), (2, 3)],[(3, 1), (3, 2), (3, 3)]]
for row in data_index_list:
    print(row)

In [ ]:
# Outputs the first key-value pair in the data_value_dict dictionary that does not contain None in the value
# The dictionary with the key being the coordinate tuple (index list element value) and the value being a list of all raster image element values corresponding to the coordinates (in index_list order)
# Example of output: (2, 3): [3, 2, 6, 7, 9]
for index, data_list in data_value_dict.items():
    if None in data_list:
        continue
    print(f"{index}: {data_list}")
    break

Once the conversion is complete, the corresponding multiple raster pixel values can be overlaid and analyzed based on these data. For example, the **pymannkendall** module is used to perform an MK trend test on the time-series meteorological raster data to obtain the Slope value of each image element:

In [ ]:
# Create an empty list template based on the number of rows and columns of any list in the data_dict dictionary
key1 = list(data_dict.keys())[0]
list1 = data_dict[key1]
rows, cols = len(list1), len(list1[0])
slope_list = [[None for _ in range(cols)] for _ in range(rows)]

# Iterate through the data_value_dict dictionary, perform an MK trend test on the lists in the dictionary,
# and add Slope values to the corresponding positions in the list templates in sequence
for index, data_list in tqdm(data_value_dict.items()):
    if None in data_list:
        continue
    i, j = index
    mkTest = mk.original_test(data_list)
    slope_list[i][j] = mkTest.slope

**rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, nodata_value)**:

- Create a raster template based on the input raster, process the raster data by replacing pixel values with the new data.

In [ ]:
rasterconvert.process_raster_data("../Pre_Year_NE/Pre_2011.tif", "../MKData/mkSlope.tif", slope_list, None)